# Background Removal

> A tool to generate the area of interest maps of images using a variety of background removal techniques.

Code was written by Nicholas M. Synovic, Oscar Yanek, and Rohan Sethi

## Setup

### Upgrade Python `pip` Tool

Upgrade the Python `pip` tool to the latest version

In [ ]:
%pip install --upgrade pip

### Install Python libaries via `pip`

Installed libraries are:

- opencv-contrib-python
- progress

In [ ]:
%pip install opencv-contrib-python progress

### Import Dependencies 

In [ ]:
from os import listdir
from os.path import join
from pathlib import PurePath

import cv2
import numpy
from numpy import ndarray
from progress.bar import Bar

### Allow Data to be Loaded From Google Drive

If you wish to load data from Google Drive, uncomment the following lines.

In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')

## Application

### Read Directory

Function to read a directory and return a list of filepaths from that directory.

In [ ]:
def readDirectory(dir: str) -> list:
    files: list = listdir(dir)
    filepaths: list = [join(dir, f) for f in files]
    return filepaths

### Triangle Threshold Background Removal

Takes a file path to an image (`imagePath`) and an output folder path (default is `./data`; `outputFolder`) as input.

<!-- It then uses the approach outlined in [1](#citations) to estimate the background threshold of the image. -->

Area of interest maps are saved in `.jpg` format in the `outputFolder` with the following scheme:

- `outputFolder`/FILENAME_triangleBackgroundRemoval`.jpg`

Where FILENAME is the original name of the file without the extension.

In [ ]:
def triangleBackgroundRemoval(imagePath: str, outputFolder: str = "data") -> None:
    imageName: str = PurePath(imagePath).with_suffix("").name + "_triangleBackgroundRemoval.jpg"
    outputPath: str = join(outputFolder, imageName)
    bins: ndarray = numpy.array([0, 51, 102, 153, 204, 255])

    image: ndarray = cv2.imread(imagePath)
    blurredImage: ndarray = cv2.GaussianBlur(image, (5, 5), 0)

    blurredImage[:, :, :] = numpy.digitize(blurredImage[:, :, :], bins, right=True) * 51
    grayBlurredImage: ndarray = cv2.cvtColor(blurredImage, cv2.COLOR_BGR2GRAY)

    ret, foreground = cv2.threshold(
        grayBlurredImage, 0, 255, cv2.THRESH_TOZERO_INV + cv2.THRESH_TRIANGLE
    )

    foreground[foreground > 0] = 255    # Convert from grayscale to black and white

    cv2.imwrite(outputPath, foreground)

### Otsu Threshold Background Removal

Takes a file path to an image (`imagePath`) and an output folder path (default is `./data`; `outputFolder`) as input.

<!-- It then uses the approach outlined in [2](#citations) to estimate the background threshold of the image. -->

Area of interest maps are saved in `.jpg` format in the `outputFolder` with the following scheme:

- `outputFolder`/FILENAME_otsuBackgroundRemoval`.jpg`

Where FILENAME is the original name of the file without the extension.

In [ ]:
def otsuBackgroundRemoval(imagePath: str, outputFolder: str = "data") -> None:
    imageName: str = PurePath(imagePath).with_suffix("").name + "_otsuBackgroundRemoval.jpg"
    outputPath: str = join(outputFolder, imageName)
    bins: ndarray = numpy.array([0, 51, 102, 153, 204, 255])

    image: ndarray = cv2.imread(imagePath)
    blurredImage: ndarray = cv2.GaussianBlur(image, (5, 5), 0)

    blurredImage[:, :, :] = numpy.digitize(blurredImage[:, :, :], bins, right=True) * 51
    grayBlurredImage: ndarray = cv2.cvtColor(blurredImage, cv2.COLOR_BGR2GRAY)

    ret, foreground = cv2.threshold(
        grayBlurredImage, 0, 255, cv2.THRESH_TOZERO_INV + cv2.THRESH_OTSU
    )

    foreground[foreground > 0] = 255    # Convert from grayscale to black and white

    cv2.imwrite(outputPath, foreground)

### Basic Background Removal

Takes a file path to an image (`imagePath`) and an output folder path (default is `./data`; `outputFolder`) as input.

<!-- It then uses the approach outlined in [3](#citations) to estimate the background threshold of the image. -->

Area of interest maps are saved in `.jpg` format in the `outputFolder` with the following scheme:

- `outputFolder`/FILENAME_basicBackgroundRemoval`.jpg`

Where FILENAME is the original name of the file without the extension.

In [ ]:
def basicBackgroundRemoval(imagePath: str, outputFolder: str = "data") -> None:
    imageName: str = PurePath(imagePath).with_suffix("").name + "_basicBackgroundRemoval.jpg"
    outputPath: str = join(outputFolder, imageName)

    image: ndarray = cv2.imread(imagePath)

    grayImage: ndarray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    baseline: ndarray = cv2.threshold(grayImage,127,255,cv2.THRESH_TRUNC)[1]
    foreground: ndarray = cv2.threshold(baseline,126,255,cv2.THRESH_BINARY_INV)[1]

    foreground[foreground > 0] = 255    # Convert from grayscale to black and white

    cv2.imwrite(outputPath, foreground)

### HSV Background Removal

Takes a file path to an image (`imagePath`) and an output folder path (default is `./data`; `outputFolder`) as input.

<!-- It then uses the approach outlined in [4](#citations) to estimate the background threshold of the image. -->

Area of interest maps are saved in `.jpg` format in the `outputFolder` with the following scheme:

- `outputFolder`/FILENAME_hsvBackgroundRemoval`.jpg`

Where FILENAME is the original name of the file without the extension.

In [ ]:
def hsvBackgroundRemoval(imagePath: str, outputFolder: str = "data") -> None:
    imageName: str = PurePath(imagePath).with_suffix("").name + "_hsvBackgroundRemoval.jpg"
    outputPath: str = join(outputFolder, imageName)

    image: ndarray = cv2.imread(imagePath)

    hsvImage = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    saturation: hsvImage = hsvImage[:,:,1]
    saturation = numpy.where(saturation < 127, 0, 1)

    colorValue: ndarray = (hsvImage[:,:,2] + 127) % 255
    colorValue = numpy.where(colorValue > 127, 1, 0)

    foreground = numpy.where(saturation + colorValue > 0, 1, 0).astype(numpy.uint8)

    foreground[foreground > 0] = 255    # Convert from grayscale to black and white

    cv2.imwrite(outputPath, foreground)

### Main Method

In [ ]:
def main() -> None:
    dir: str = input("Image directory to analyze: ")
    imagePaths: list = readDirectory(dir)
    
    with Bar(
        "Creating saliency maps of PascalVOC images...", max=len(imagePaths)
    ) as bar:
        imagePath: str
        for imagePath in imagePaths:
            triangleBackgroundRemoval(imagePath)
            otsuBackgroundRemoval(imagePath)
            basicBackgroundRemoval(imagePath)
            hsvBackgroundRemoval(imagePath)
            bar.next()


if __name__ == "__main__":
    main()